In [1]:

import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
#from config import db_password

CREATE DATASET

In [2]:

# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'caf3rac3',
    database='postgres'
    )
cursor=connection.cursor()

In [3]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
All_NBA_Unpivoted_df.head()

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019


In [4]:

#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [5]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [6]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,1978,Zaid Abdul-Aziz,PF-C,31,TOT,16,0,158.0,11.4,0.449,...,19.0,31.0,50.0,10.0,3.0,3.0,14.0,29.0,63.0,1977
1,1978,Zaid Abdul-Aziz,C,31,BOS,2,0,24.0,11.5,0.279,...,6.0,9.0,15.0,3.0,1.0,1.0,3.0,4.0,8.0,1977
2,1978,Mel Bennett,PF,23,IND,31,0,285.0,7.6,0.367,...,49.0,44.0,93.0,22.0,11.0,7.0,30.0,54.0,74.0,1977
3,1978,Mack Calvin,PG,30,DEN,77,0,988.0,14.0,0.551,...,11.0,73.0,84.0,148.0,46.0,5.0,108.0,87.0,467.0,1977
4,1978,Austin Carr,SG,29,CLE,82,0,2186.0,13.6,0.484,...,76.0,111.0,187.0,225.0,68.0,19.0,146.0,168.0,1011.0,1977


In [7]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [8]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [9]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [10]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))

In [11]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()
Positions_List

array(['PF-C', 'C', 'PF', 'PG', 'SG', 'SF', 'SF-SG', 'SG-SF', 'C-PF',
       'SF-PF', 'PG-SG', 'SG-PG', 'PF-SF', 'PG-SF', 'SG-PF', 'C-SF'],
      dtype=object)

In [12]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [13]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )

In [14]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)

In [15]:
# drop year  and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_start'],axis=1)

Season_Stats_ML_DF.columns

Index(['Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS_pct', '3PAr', 'FTr', 'ORB_pct',
       'DRB_pct', 'TRB_pct', 'AST_pct', 'STL_pct', 'BLK_pct', 'TOV_pct',
       'USG_pct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG_pct', '3P', '3PA', '3P_pct', '2P', '2PA', '2P_pct',
       'eFG_pct', 'FT', 'FTA', 'FT_pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'AllNBA_Center', 'AllNBA_Forward',
       'AllNBA_Guard', 'Tm_1st', 'Tm_2nd', 'Tm_3rd', 'is_Guard', 'is_Forward',
       'is_Center'],
      dtype='object')

In [16]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [17]:
Season_Stats_ML_DF['3PAr'] = Season_Stats_ML_DF['3PAr'].fillna(int(0))
Season_Stats_ML_DF['3P'] = Season_Stats_ML_DF['3P'].fillna(int(0))
Season_Stats_ML_DF['3PA'] = Season_Stats_ML_DF['3PA'].fillna(int(0))
Season_Stats_ML_DF['3P_pct'] = Season_Stats_ML_DF['3P_pct'].fillna(int(0))


In [18]:
#make dummies for team
#Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm'])
#I don't like this lol

In [19]:
# Fill null values with zero
for col in list(Season_Stats_ML_DF.columns):
    Season_Stats_ML_DF[col] = Season_Stats_ML_DF[col].fillna(int(0))

ALL NBA CENTER MACHINE LEARNING SECTION

In [20]:
#Create Dataset for X & y, make sure it is only centers...we don't need guard or forward data her

# implement random oversampling
from imblearn.under_sampling import ClusterCentroids


X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
y = y["AllNBA_Center"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#undersampling
cc = ClusterCentroids(random_state=78)
X_train, y_train = cc.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,462,500
Actual 1,3,22


Accuracy Score : 0.49037487335359675
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.48      0.65       962
         1.0       0.04      0.88      0.08        25

    accuracy                           0.49       987
   macro avg       0.52      0.68      0.36       987
weighted avg       0.97      0.49      0.63       987



In [21]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
center_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
center_feature_importance

[(0.6586894586894586, 'WS'),
 (0.08792858792858796, 'FTA'),
 (0.05993653337403343, 'BLK_pct'),
 (0.05860805860805862, '3P_pct'),
 (0.0445054945054945, 'ORB_pct'),
 (0.023487773487773457, 'TRB_pct'),
 (0.021978021978021973, 'TOV'),
 (0.02142857142857144, 'PER'),
 (0.01785714285714286, 'BPM'),
 (0.005580357142857144, 'USG_pct'),
 (0.0, 'eFG_pct'),
 (0.0, 'VORP'),
 (0.0, 'TS_pct'),
 (0.0, 'TRB'),
 (0.0, 'TOV_pct'),
 (0.0, 'STL'),
 (0.0, 'SPG'),
 (0.0, 'RPG'),
 (0.0, 'PTS'),
 (0.0, 'PPG'),
 (0.0, 'PF'),
 (0.0, 'OWS'),
 (0.0, 'ORB'),
 (0.0, 'OBPM'),
 (0.0, 'MP'),
 (0.0, 'GS'),
 (0.0, 'G'),
 (0.0, 'FTr'),
 (0.0, 'FT_pct'),
 (0.0, 'FT'),
 (0.0, 'FG_pct'),
 (0.0, 'FGA'),
 (0.0, 'FG'),
 (0.0, 'DWS'),
 (0.0, 'DRB_pct'),
 (0.0, 'DRB'),
 (0.0, 'DBPM'),
 (0.0, 'BLK'),
 (0.0, 'Age'),
 (0.0, 'AST_pct'),
 (0.0, 'AST'),
 (0.0, '3PAr'),
 (0.0, '3PA'),
 (0.0, '3P'),
 (0.0, '2P_pct'),
 (0.0, '2PA'),
 (0.0, '2P')]

ALL NBA GUARD MACHINE LEARNING SECTION

In [22]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
y = y["AllNBA_Guard"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#undersampling
cc = ClusterCentroids(random_state=78)
X_train, y_train = cc.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1113,788
Actual 1,7,54


Accuracy Score : 0.5948012232415902
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.59      0.74      1901
         1.0       0.06      0.89      0.12        61

    accuracy                           0.59      1962
   macro avg       0.53      0.74      0.43      1962
weighted avg       0.96      0.59      0.72      1962



In [23]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
guard_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
guard_feature_importance

[(0.6931697950546651, 'PER'),
 (0.07959019429107139, 'FT_pct'),
 (0.0486301369863014, 'BLK_pct'),
 (0.04676564156945916, 'G'),
 (0.037354100519555546, 'WS'),
 (0.02417763157894736, 'AST_pct'),
 (0.021875, '2P_pct'),
 (0.01875, 'STL'),
 (0.01218926553672321, 'FT'),
 (0.0109375, '3PAr'),
 (0.005104166666666654, 'MP'),
 (0.001456567796610135, '2P'),
 (0.0, 'eFG_pct'),
 (0.0, 'VORP'),
 (0.0, 'USG_pct'),
 (0.0, 'TS_pct'),
 (0.0, 'TRB_pct'),
 (0.0, 'TRB'),
 (0.0, 'TOV_pct'),
 (0.0, 'TOV'),
 (0.0, 'SPG'),
 (0.0, 'RPG'),
 (0.0, 'PTS'),
 (0.0, 'PPG'),
 (0.0, 'PF'),
 (0.0, 'OWS'),
 (0.0, 'ORB_pct'),
 (0.0, 'ORB'),
 (0.0, 'OBPM'),
 (0.0, 'GS'),
 (0.0, 'FTr'),
 (0.0, 'FTA'),
 (0.0, 'FG_pct'),
 (0.0, 'FGA'),
 (0.0, 'FG'),
 (0.0, 'DWS'),
 (0.0, 'DRB_pct'),
 (0.0, 'DRB'),
 (0.0, 'DBPM'),
 (0.0, 'BPM'),
 (0.0, 'BLK'),
 (0.0, 'Age'),
 (0.0, 'AST'),
 (0.0, '3P_pct'),
 (0.0, '3PA'),
 (0.0, '3P'),
 (0.0, '2PA')]

ALL NBA FORWARD MACHINE LEARNING SECTION

In [24]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her


X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
y = y["AllNBA_Forward"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#undersampling
cc = ClusterCentroids(random_state=78)
X_train, y_train = cc.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1458,485
Actual 1,6,42


Accuracy Score : 0.7533902561526871
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      0.75      0.86      1943
         1.0       0.08      0.88      0.15        48

    accuracy                           0.75      1991
   macro avg       0.54      0.81      0.50      1991
weighted avg       0.97      0.75      0.84      1991



In [25]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
forward_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
forward_feature_importance

[(0.7565821256038647, 'PER'),
 (0.08828379517979998, 'BPM'),
 (0.057520052836650744, 'G'),
 (0.027176220806794056, 'DWS'),
 (0.024178843511927058, 'FG_pct'),
 (0.012650389242745917, 'RPG'),
 (0.012475901115259832, 'TOV_pct'),
 (0.009554140127388535, 'PTS'),
 (0.008492569002123142, '3P'),
 (0.0030859625734459078, 'PPG'),
 (0.0, 'eFG_pct'),
 (0.0, 'WS'),
 (0.0, 'VORP'),
 (0.0, 'USG_pct'),
 (0.0, 'TS_pct'),
 (0.0, 'TRB_pct'),
 (0.0, 'TRB'),
 (0.0, 'TOV'),
 (0.0, 'STL'),
 (0.0, 'SPG'),
 (0.0, 'PF'),
 (0.0, 'OWS'),
 (0.0, 'ORB_pct'),
 (0.0, 'ORB'),
 (0.0, 'OBPM'),
 (0.0, 'MP'),
 (0.0, 'GS'),
 (0.0, 'FTr'),
 (0.0, 'FT_pct'),
 (0.0, 'FTA'),
 (0.0, 'FT'),
 (0.0, 'FGA'),
 (0.0, 'FG'),
 (0.0, 'DRB_pct'),
 (0.0, 'DRB'),
 (0.0, 'DBPM'),
 (0.0, 'BLK_pct'),
 (0.0, 'BLK'),
 (0.0, 'Age'),
 (0.0, 'AST_pct'),
 (0.0, 'AST'),
 (0.0, '3P_pct'),
 (0.0, '3PAr'),
 (0.0, '3PA'),
 (0.0, '2P_pct'),
 (0.0, '2PA'),
 (0.0, '2P')]